In [1]:
import os
from tqdm import tqdm
from data_handler import DataHandler
from evaluation import get_evaluation
from image_processor import ImageProcessor
from model_handler import get_model

/home/baojiedama/miniconda3/envs/rope/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/baojiedama/miniconda3/envs/rope/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
def main(model_name, model_size, model_path, device_map, data_base_path, output_base_path):
    # Initialize model handler, data handler, and image processor
    model = get_model(model_name, model_size, model_path, device_map)
    data_handler = DataHandler(data_base_path)
    image_processor = ImageProcessor()

    # Evaluation settings and data types
    settings = ["default", "student-forcing", "teacher-forcing", "single"]
    data_types = ["train", "validation"]

    # Iterate through each setting and data type
    for setting in tqdm(settings, desc="Settings"):
        for data_type in tqdm(data_types, desc=f"Data Types for {setting}", leave=False):
            # Load the data for the current data type
            data = data_handler.load_data(data_type)
            
            # Get the evaluation class for the current setting
            evaluation_class = get_evaluation(setting)
            
            # Define the output path
            output_path = os.path.join(output_base_path, model_name+model_size, setting, data_type)
            
            # Process each data file
            for file_name, entries in tqdm(data.items(), desc=f"Files for {data_type}", leave=False):
                processed_data = []
                photo2answer = {}
                acc_list = [0, 0, 0, 0, 0]
                evaluator = evaluation_class(model, image_processor, data_base_path, data_type)

                try:
                    entries = entries[:5]
                    for entry in tqdm(entries, desc=f"Entries for {file_name}", leave=False):
                        evaluator.process_entry(entry, acc_list, photo2answer)
                        processed_data.append(entry)

                        # Save partial results after processing each entry
                        metrics = evaluator.calculate_metrics(acc_list)
                        data_handler.save_partial_results(processed_data, metrics, photo2answer, output_path, file_name)
                    # data_handler.save_final_results(processed_data, metrics, photo2answer, output_path, file_name)
                except Exception as e:
                    print(f"Error processing file {file_name} in setting {setting}, data type {data_type}: {e}")

In [3]:
# Define parameters
model_name = "llava-hf"  # 你的LLaVA handler注册的名称
model_size = "7b"
model_path = "/root/.cache/huggingface/hub/models--llava-hf--llava-1.5-7b-hf/snapshots/6ceb2ed33cb8f107a781c431fe2e61574da69369"  # 你本地模型目录或repo名
device_map = "cuda"  # 一般直接"cuda"，多卡可以 balanced
data_base_path = "/home/baojiedama/ROPE/"  # 指向 database 目录
output_base_path = "/home/baojiedama/moh-output-experiments"  # 新建输出目录

# Run the main function
main(model_name, model_size, model_path, device_map, data_base_path, output_base_path)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]